In [1]:
import cv2 as cv
import numpy as np
import json
import os
import matplotlib.pyplot as plt
from tqdm import tqdm


In [2]:
COLORS = [(0, 255, 0), (255, 0, 0)]
CLASSES = ['pool', 'lack_of_fusion']

In [3]:
def label2mask(json_file, root='mydata', show=False):
    """According to the json file, create a mask for the image.

    Save mask file with same name in {root}/seg/ folder. 

    Save into .jpg format.  

    Args:
        json_file (str): Json file path
        show (bool, optional): Show the mask. Defaults to False.
    """
    os.makedirs(os.path.join(root, 'seg'), exist_ok=True)
    try:
        with open(json_file, 'r') as f:
            data = json.load(f)
    except:
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
    img = np.zeros(
        (data['imageHeight'], data['imageWidth'], 3), dtype=np.uint8)
    save_path = os.path.join(root, 'seg', os.path.basename(
        json_file).split('.')[0] + '.png')
    labels = data['shapes']
    for label in labels:
        points = np.array(label['points'], dtype=np.int32)
        cv.fillPoly(img, [points], COLORS[CLASSES.index(label['label'])])
    cv.imwrite(save_path, img)
    if show:
        plt.imshow(img)
        plt.show()


def convert_img(root='mydata'):
    """Convert images into .jpg format.

    Args:
        root (str, optional): _description_. Defaults to 'mydata'.
    """
    os.makedirs(os.path.join(root, 'images'), exist_ok=True)
    for img_file in tqdm(os.listdir(os.path.join(root, 'bmp'))):
        img = cv.imread(os.path.join(root, 'bmp', img_file))
        cv.imwrite(os.path.join(
            root, 'images', img_file.split('.')[0] + '.jpg'), img)


# for json_file in tqdm(os.listdir('mydata/Json')):
#     label2mask(os.path.join('mydata/Json', json_file))
# convert_img()

In [4]:
def show_images(imgs, num_rows, num_cols, titles=None, scale=1.5):
    """ Display a list of images.
    
    Args:
        imgs (list): List of images
        num_rows (int): Number of rows
        num_cols (int): Number of columns
        titles (list, optional): List of titles. Defaults to None.
        scale (float, optional): Scale. Defaults to 1.5.
        
    Returns:
        list: List of axes
    """
    figsize = (num_cols * scale, num_rows * scale)
    _, axes = plt.subplots(num_rows, num_cols, figsize=figsize)
    axes = axes.flatten()
    for i, (ax, img) in enumerate(zip(axes, imgs)):
        ax.imshow(img.permute(1, 2, 0))
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
        if titles:
            ax.set_title(titles[i])
    return axes